In [44]:
import numpy as np
import pandas as pd
data = pd.read_csv("Train (1).csv")
data.head()

,tweet_id,safe_text,label,agreement
0,CL1KWCMY,Me &amp; The Big Homie meanboy3000 #MEANBOY #M...,0.0,1.0
1,E3303EME,I'm 100% thinking of devoting my career to pro...,1.0,1.0
2,M4IVFSMS,"#whatcausesautism VACCINES, DO NOT VACCINATE Y...",-1.0,1.0
3,1DR6ROZ4,I mean if they immunize my kid with something ...,-1.0,1.0
4,J77ENIIE,Thanks to <user> Catch me performing at La Nui...,0.0,1.0


In [45]:
data.agreement.unique()

array([1.        , 0.66666667, 0.33333333,        nan])

In [46]:
data = data.drop(columns=['tweet_id'])
data.head()

,safe_text,label,agreement
0,Me &amp; The Big Homie meanboy3000 #MEANBOY #M...,0.0,1.0
1,I'm 100% thinking of devoting my career to pro...,1.0,1.0
2,"#whatcausesautism VACCINES, DO NOT VACCINATE Y...",-1.0,1.0
3,I mean if they immunize my kid with something ...,-1.0,1.0
4,Thanks to <user> Catch me performing at La Nui...,0.0,1.0


In [47]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001 entries, 0 to 10000
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   safe_text  10001 non-null  object 
 1   label      10000 non-null  float64
 2   agreement  9999 non-null   float64
dtypes: float64(2), object(1)
memory usage: 234.5+ KB


In [48]:
data = data.dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9999 entries, 0 to 10000
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   safe_text  9999 non-null   object 
 1   label      9999 non-null   float64
 2   agreement  9999 non-null   float64
dtypes: float64(2), object(1)
memory usage: 312.5+ KB


In [49]:
data.label.unique()

array([ 0.,  1., -1.])

In [50]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, f1_score
import nltk
from nltk.corpus import stopwords

In [51]:
data['safe_text']= data['safe_text'].apply(lambda x: x.replace('&amp;' ,''))
data.head()
data['safe_text'] = data['safe_text'].apply(str)
data.safe_text = data.safe_text.apply(str.lower)

In [52]:
data.safe_text = data.safe_text.apply(lambda x: x.strip().strip('.'))

In [53]:
test_data = pd.read_csv('Test.csv')
test_data.head()

,tweet_id,safe_text
0,00BHHHP1,<user> <user> ... &amp; 4 a vaccine given 2 he...
1,00UNMD0E,Students starting school without whooping coug...
2,01AXPTJF,"I'm kinda over every ep of <user> being ""rippe..."
3,01HOEQJW,How many innocent children die for lack of vac...
4,01JUKMAO,"CDC eyeing bird flu vaccine for humans, though..."


In [54]:
test_data['safe_text'] = test_data['safe_text'].apply(str)
test_data.safe_text = test_data.safe_text.apply(str.lower)
test_data.safe_text = test_data.safe_text.apply(lambda x: x.replace('&amp;', ''))
test_data.safe_text = test_data.safe_text.apply(lambda x: x.strip().strip('.'))

In [55]:
def clean_stop(new_data):
    stop_words = set(stopwords.words('english'))
    #df['text_clean'] = df['safe_text'].apply(lambda s: " ".join([w for w in s.lower().split() if w not in stop_words and not w.startswith('<')]))
    new_data['text_clean'] = new_data['safe_text'].apply(lambda s: " ".join([w for w in s.lower().split() if not w.startswith('<')]))
    new_data['text_clean'] = new_data['text_clean'].apply(lambda s: " ".join([w for w in s.lower().split() if not w.startswith('#')]))

In [56]:
new_data = pd.concat([data, test_data], axis=0)
clean_stop(new_data)


In [57]:
new_data.head()

,safe_text,label,agreement,tweet_id,text_clean
0,me the big homie meanboy3000 #meanboy #mb #mb...,0.0,1.0,NaN,me the big homie meanboy3000 @ stegman st.
1,i'm 100% thinking of devoting my career to pro...,1.0,1.0,NaN,i'm 100% thinking of devoting my career to pro...
2,"#whatcausesautism vaccines, do not vaccinate y...",-1.0,1.0,NaN,"vaccines, do not vaccinate your child"
3,i mean if they immunize my kid with something ...,-1.0,1.0,NaN,i mean if they immunize my kid with something ...
4,thanks to <user> catch me performing at la nui...,0.0,1.0,NaN,thanks to catch me performing at la nuit nyc 1...


In [61]:
train = new_data[~new_data.label.isnull()]
test = new_data[new_data['label'].isnull()]
train.head()
test.head()

,safe_text,label,agreement,tweet_id,text_clean
0,<user> <user> ... 4 a vaccine given 2 healthy...,NaN,NaN,00BHHHP1,"... 4 a vaccine given 2 healthy peeps, fda thi..."
1,students starting school without whooping coug...,NaN,NaN,00UNMD0E,students starting school without whooping coug...
2,"i'm kinda over every ep of <user> being ""rippe...",NaN,NaN,01AXPTJF,"i'm kinda over every ep of being ""ripped from ..."
3,how many innocent children die for lack of vac...,NaN,NaN,01HOEQJW,how many innocent children die for lack of vac...
4,"cdc eyeing bird flu vaccine for humans, though...",NaN,NaN,01JUKMAO,"cdc eyeing bird flu vaccine for humans, though..."


In [60]:
train.label += 1
train.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,safe_text,label,agreement,tweet_id,text_clean
0,me the big homie meanboy3000 #meanboy #mb #mb...,1.0,1.0,NaN,me the big homie meanboy3000 @ stegman st.
1,i'm 100% thinking of devoting my career to pro...,2.0,1.0,NaN,i'm 100% thinking of devoting my career to pro...
2,"#whatcausesautism vaccines, do not vaccinate y...",0.0,1.0,NaN,"vaccines, do not vaccinate your child"
3,i mean if they immunize my kid with something ...,0.0,1.0,NaN,i mean if they immunize my kid with something ...
4,thanks to <user> catch me performing at la nui...,1.0,1.0,NaN,thanks to catch me performing at la nuit nyc 1...


In [63]:
x_train = train.text_clean.values
y_train =train.label.values
x_test = test.text_clean.values


tvect = TfidfVectorizer(min_df=3, max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

cvect = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), stop_words = 'english')

In [66]:
tvect.fit(new_data['text_clean'].values.tolist())
cvect.fit(new_data['text_clean'].values.tolist())

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 3), preprocessor=None, stop_words='english',
                strip_accents=None, token_pattern='\\w{1,}', tokenizer=None,
                vocabulary=None)

In [67]:
x_train_tf = tvect.transform(x_train)
x_test_tf = tvect.transform(x_test)
x_train_cv = cvect.transform(x_train)
x_test_cv = cvect.transform(x_test)

In [69]:
def predictions(preds):
    final_predictions = []
    for pred in preds:
        argmax = pred.argmax()
        if argmax == 0: final_predictions.append( -1*pred[0])
        elif argmax == 1: final_predictions.append( 0 )
        else: final_predictions.append( pred[2] )
    return final_predictions
def rmse(true, pred):
    return np.sqrt(mean_squared_error(true, pred))